In [ ]:
import requests
import sseclient
import json

from IPython.display import display, Markdown

In [ ]:
course_description = """
Robotics is revolutionizing industries, from manufacturing to healthcare, 
by combining mechanical systems, electronics, and artificial intelligence. 
This Intro to Robotics course provides a foundational understanding of 
how robots work, their applications, and the core technologies behind them.

You’ll explore key topics such as robot kinematics, sensors, actuators, 
programming, and AI-driven automation. Through hands-on projects and 
simulations, you’ll learn how to design, build, and program simple robotic
systems.

This course is ideal for beginners, students, and professionals looking
to gain insights into robotic hardware, software, and real-world applications. 
By the end of the course, you’ll have a solid understanding of how robots 
perceive their environment, make decisions, and interact with the physical world.

Join us to take your first step into the exciting field of robotics!
"""

# Course Outline

In [ ]:
url = "http://localhost:8000/stream_outline"
data = {
    "title": "Introduction to Robotics",
    "description": course_description
}

# 1) Start the streaming request
response = requests.post(url, json=data, stream=True)

# 2) Wrap the response with SSEClient
client = sseclient.SSEClient(response.iter_lines())

session_id = None

# 3) Iterate over events
for event in client.events():
    if event.data:  # If there's data in the SSE event
        cleaned = event.data.split("data: ")
        
        try:
            parsed1 = json.loads(cleaned[0])
            parsed2 = json.loads(cleaned[1])

            # If we see a key "session_id", we can store it
            if "session_id" in parsed1:
                session_id = parsed1["session_id"]
                print("Received session_id:", session_id, "\n")
            
            display(Markdown(parsed2["generate_outline"]["outline"]))
            
        except json.JSONDecodeError:
            print("Raw SSE string:", event.data)

# Course Lessons

In [ ]:
url = f"http://localhost:8000/stream_lessons/{session_id}"

response = requests.get(url, stream=True)
client = sseclient.SSEClient(response)

for event in client.events():
    if event.data:
        # Typically {"lessons": "..."}
        parsed = json.loads(event.data)
        print("Lessons SSE:", parsed)

In [ ]:
display(Markdown(parsed["generate_lessons"]["lessons"]))

# Refine Lessons

In [ ]:
url = "http://localhost:8000/stream_refine_lessons"
data = {
    "session_id": session_id,
    "feedback": "Please include more hands-on examples"
}

response = requests.post(url, json=data, stream=True)
client = sseclient.SSEClient(response)

for event in client.events():
    if event.data:
        # Typically {"refined_lessons": "..."}
        parsed = json.loads(event.data)
        print("Refined SSE:", parsed)

In [ ]:
display(Markdown(parsed["refine_lessons"]["lessons"]))